# Inference using trained models

In [1]:
!pip install simpletransformers
import pandas as pd
import numpy as np
import os
import re
import json
import torch
import matplotlib.pyplot as plt
from simpletransformers.classification import MultiLabelClassificationModel
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

import pickle


##### GOOGLE COLAB CODE, COMMENT IT WHEN RUNNING LOCALLY    ###########
# Mount your Google Drive to Collaboratory
from google.colab import drive 
drive.mount('/content/gdrive')
###### END GOOGLE COLAB CODE ########
DATA_PATH = "gdrive/My Drive/temp/HTC/")

     |████████████████████████████████| 225kB 7.2MB/s 
     |████████████████████████████████| 7.4MB 7.5MB/s 
     |████████████████████████████████| 317kB 41.8MB/s 
     |████████████████████████████████| 1.8MB 60.5MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 51kB 9.8MB/s 
     |████████████████████████████████| 2.9MB 56.8MB/s 
     |████████████████████████████████| 1.3MB 61.1MB/s 
     |████████████████████████████████| 133kB 64.4MB/s 
     |████████████████████████████████| 102kB 15.7MB/s 
     |████████████████████████████████| 6.9MB 52.0MB/s 
     |████████████████████████████████| 4.5MB 38.4MB/s 
     |████████████████████████████████| 163kB 59.7MB/s 
     |████████████████████████████████| 112kB 61.5MB/s 
     |████████████████████████████████| 133kB 62.0MB/s 
     |████████████████████████████████| 102kB 16.4MB/s 
     |████████████████████████████████| 1.1MB 51.7MB/s 
     |████████████████████████████████| 890kB 50.4MB/

Mounted at /content/gdrive


## test inference using only one model

In [ ]:


model_path = DATA_PATH + "/models_final/classifier_Category"
model = MultiLabelClassificationModel("xlnet", model_path, use_cuda=False)


train_df = merged_df.copy().dropna(subset=["Text", "Category"])
test_df = train_df.sample(frac=0.01)
text_list = test_df["Text"].tolist()
golden_labels = test_df["Category"]


with open(model_path+"/encoder.pkl", 'rb') as f:
            one_hot_encoder = pickle.load(f)
predictions_oh, _ = model.predict(text_list)
golden_labels_oh = one_hot_encoder.transform(golden_labels)
categories = one_hot_encoder.classes_

print(classification_report(golden_labels_oh, predictions_oh, target_names=categories))


                                      precision    recall  f1-score   support

                      Data Retention       1.00      0.50      0.67         2
                       Data Security       1.00      1.00      1.00         6
                        Do Not Track       0.00      0.00      0.00         0
          First Party Collection/Use       1.00      0.88      0.93         8
International and Specific Audiences       1.00      1.00      1.00         2
                Introductory/Generic       0.83      0.71      0.77         7
                       Policy Change       1.00      1.00      1.00         2
                Practice not covered       0.33      0.50      0.40         2
         Privacy contact information       1.00      1.00      1.00         4
      Third Party Sharing/Collection       0.79      1.00      0.88        11
      User Access, Edit and Deletion       1.00      1.00      1.00         4
                 User Choice/Control       1.00      0.75     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Hiearchical inference using all models

In [2]:

    
class HTC_classifier():
    def __init__(self, ephocs=1, model_type='xlnet', model_path='xlnet-base-cased', output_path=None):
        self.one_hot = MultiLabelBinarizer()
        self.use_cuda = torch.cuda.is_available()
        self.ephocs = ephocs
        self.model_type = model_type
        self.model_path = model_path
        self.output_path = output_path
        self.load_model()
        print("self.use_cuda: ", self.use_cuda)

    def one_hot_encoder(self, labels, fit=False):
        if(fit):
            return self.one_hot.fit_transform(labels)
        return self.one_hot.transform(labels)


    def one_hot_to_text(self, one_hot_matrix):
        return self.one_hot.inverse_transform(one_hot_matrix)


    def fit(self, text_series, labels):
        labels = self.one_hot_encoder(labels, True)
        print("fitting model, artifacts will be saved to: ", self.output_path)
        self.model = MultiLabelClassificationModel(self.model_type, self.model_path, num_labels=np.array(labels).shape[1] ,use_cuda=self.use_cuda,
                                                   args={'reprocess_input_data': True, 'output_dir' :self.output_path , 'overwrite_output_dir': True,
                                                         'num_train_epochs': self.ephocs, 'save_steps' :0})
        train_df = pd.DataFrame({"text": text_series, "labels": labels.tolist()})
        self.model.train_model(train_df)
        self.save_enoder()
        return self.output_path

    def save_enoder(self):
        # Serialize both the pipeline and binarizer to disk.
        encoder_filename = self.output_path +"/encoder.pkl"
        with open(encoder_filename, 'wb') as f:
            pickle.dump((self.one_hot), f)
        print("model saved to: ", self.output_path)

    def load_model(self):
        # Hydrate the serialized objects.
        encoder_filename = self.model_path + "/encoder.pkl"
        print("loading model from: ", self.model_path)
        try:
            with open(encoder_filename, 'rb') as f:
                self.one_hot = pickle.load(f)
            self.model = MultiLabelClassificationModel('xlnet', self.model_path,use_cuda=self.use_cuda)
        except Exception as e:
            print("error:", e)
            print("couldn't load models from disk")

    def predict(self, text_series):
        text_list = text_series.tolist()
        predictions, _ = self.model.predict(text_list)
        return self.one_hot_to_text(np.array(predictions))

    def evaluate(self, text_series, golden_labels):
        predictions = self.predict(text_series)
        predictions_oh = self.one_hot_encoder(predictions)
        golden_labels_oh = self.one_hot_encoder(golden_labels)
        categories = self.one_hot.classes_
        print(classification_report(golden_labels_oh, predictions_oh, target_names=categories))




In [5]:
import gc


class Hierarchical_classifier():
  def __init__(self, models_path):
    self.models_path = models_path
    self.model_prefix = "classifier_"
    self.category_atttributes_map = {'Introductory/Generic': [],
                                'First Party Collection/Use': ['Action First-Party', 'Personal Information Type', 'Purpose', 'User Type','Does/Does Not', 'Identifiability', 'Choice Type','Collection Mode', 'Choice Scope'],
                                'User Choice/Control': ['Choice Type', 'Choice Scope','Purpose', 'Personal Information Type', 'User Type'],
                                'International and Specific Audiences': ['Audience Type'],
                                'Third Party Sharing/Collection': ['Action Third Party', 'Third Party Entity', 'Personal Information Type', 'Purpose', 'User Type','Does/Does Not', 'Identifiability', 'Choice Type','Collection Mode', 'Choice Scope'],
                                'Data Security': ['Security Measure'],
                                'Practice not covered': [],
                                'User Access, Edit and Deletion': ['Access Scope', 'User Type', 'Access Type'],
                                'Privacy contact information': [],
                                'Policy Change': ['Change Type', 'Notification Type', 'User Choice'],
                                'Data Retention': ['Retention Period', 'Retention Purpose', 'Personal Information Type'],
                                'Do Not Track': ['Do Not Track policy']
  }

  def load_model(self, model_name):
    model_id = "classifier_"+"_".join(model_name.split())
    model_path = self.models_path + "/"+model_id
    return HTC_classifier(model_type='xlnet', model_path=model_path)

  def predict_using_model(self, attribute, text_list):
    results = self.load_model(attribute).predict(pd.Series(text_list))
    gc.collect()    #IMPORTANT: this prevents a mem leak in transformers library: https://github.com/huggingface/transformers/issues/1742
    return results
  
  def predict(self, text_list):
    text_results_map = {}
    cat_texts_map = {}
    category_labels = self.predict_using_model('Category', text_list)
    print("category_labels: ", category_labels)
    for i, text in enumerate(text_list):
      categories = category_labels[i]
      for category in categories:
        if(category not in cat_texts_map):
          cat_texts_map[category] = []
        cat_texts_map[category].append(text)
    text_result_map = {}
    for cat, texts in cat_texts_map.items():
      attributes = self.category_atttributes_map[cat]
      print("cat: {} - attributes: {}".format(cat, attributes))
      for i, text in enumerate(texts):
        # make sure we add category to text_result_map even if no attributes
        if(text not in text_result_map):
            text_result_map[text] = {}
        text_result_map[text][cat] = {}
      for attribute in attributes:
        print("attribute: ", attribute)
        attribute_labels = self.predict_using_model(attribute, texts)
        for i, text in enumerate(texts):
          text_result_map[text][cat][attribute] = attribute_labels[i]
    results = []
    for text in text_list:
      # make sure results maintains same order has text
      if(text in text_result_map):
        results.append(text_result_map[text])
      else:
        results.append({})
    return results



In [6]:
%%time
hierarchical_classifier = Hierarchical_classifier(DATA_PATH+"/models_final")
text_list = ['subscribing to our digital subscription, the Post-Gazette will ask for your credit card information', 'from outside sources that may include', 'We may also use other industry standard technologies like pixel tags and web beacons to track your use of our Website pages and promotions,', 'collect certain non-personally identifiable information when you visit our web pages or use our applications', 'we collect when you are signed in to Google',
             'This Privacy Policy will remain in full force and effect, even if your use of or participation in the Site or any particular service, feature, function or promotional activity offered through the Site terminates, expires, ceases, is suspended or deactivated for any reason. If you do not agree with this Privacy Policy, please do not participate in the Site or any features, activities or services offered through the Site.',
             'If you are located outside of the U.S., the European Economic Area and Switzerland, you use this Site at your own risk and initiative and you, not us, are responsible for compliance with any applicable local and national laws. If this Site is part of the Flux platform, by using this Site, participating in any Site activities, and/or providing us with your Personal Information and Other Information, you (i) consent to the transfer and/or processing of any Information you provide to the Site and to Flux, (ii) acknowledge that U.S. law may provide a lower standard of protection for personal data than the laws of your location and (iii) understand that this Site and Flux will collect, transfer, store, process and/or deal with your Information in accordance with this Privacy Policy and the Flux Privacy Policy and U.S. law. Consequently, to the full extent permitted by law, you hereby waive any claims relating to the processing of your Personal Information or Other Information in accordance with this Privacy Policy that may arise under the laws and regulations that apply to you in or of any other country or jurisdiction.']
predictions = hierarchical_classifier.predict(text_list)
print(predictions)




loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_Category
self.use_cuda:  True



category_labels:  [('First Party Collection/Use',), (), ('First Party Collection/Use',), ('First Party Collection/Use',), ('First Party Collection/Use',), ('Introductory/Generic', 'User Choice/Control'), ('International and Specific Audiences',)]
cat: First Party Collection/Use - attributes: ['Action First-Party', 'Personal Information Type', 'Purpose', 'User Type', 'Does/Does Not', 'Identifiability', 'Choice Type', 'Collection Mode', 'Choice Scope']
attribute:  Action First-Party
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_Action_First-Party
self.use_cuda:  True



attribute:  Personal Information Type
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_Personal_Information_Type
self.use_cuda:  True



attribute:  Purpose
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_Purpose
self.use_cuda:  True



attribute:  User Type
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_User_Type
self.use_cuda:  True


...


attribute:  Purpose
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_Purpose
self.use_cuda:  True



attribute:  Personal Information Type
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_Personal_Information_Type
self.use_cuda:  True



attribute:  User Type
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_User_Type
self.use_cuda:  True



cat: International and Specific Audiences - attributes: ['Audience Type']
attribute:  Audience Type
loading model from:  gdrive/My Drive/temp/HTC/models_final/classifier_Audience_Type
self.use_cuda:  True



[{'First Party Collection/Use': {'Action First-Party': ('Collect on website',), 'Personal Information Type': ('Financial',), 'Purpose': ('Basic service/feature',), 'User Type': (), 'Does/Does Not': ('Does',), 'Identifiability': (), 'Choice Type': ('Dont use service/feature',), 'Collection Mode': ('Explicit',), 'Choice Scope': ('Both',)}}, {}, {'First Party Collection/Use': {'Action First-Party': ('Collect on website',), 'Personal Information Type': ('Cookies and tracking elements',), 'Purpose': ('Analytics/Research',), 'User Type': ('not-selected',), 'Does/Does Not': ('Does',), 'Identifiability': (), 'Choice Type': ('Unspecified',), 'Collection Mode': ('Implicit',), 'Choice Scope': ('not-selected',)}}, {'First Party Collection/Use': {'Action First-Party': ('Collect on website',), 'Personal Information Type': ('Unspecified',), 'Purpose': ('Unspecified',), 'User Type': ('not-selected',), 'Does/Does Not': ('Does',), 'Identifiability': (), 'Choice Type': ('Dont use service/feature',), 'Co

### save sentences to CSV

In [8]:
pd.Series(text_list).to_csv(DATA_PATH+"/output/segments.csv", header=False, index=False)